In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
destination_table = "data_acm.marketo_campaign_response_data_processed"

In [0]:
marketo_campaignresponsedata = spark.sql("""
SELECT sends.countryCode,sends.deliveryLogId as broadLogId,
       sends.deliveryID,sends.contactpersonOhubId,sends.operatorOhubId,sends.campaignLabel,sends.deliveryLabel as waveName, sends.sentDate as sendDate,
       COUNT(DISTINCT opens.trackingId) as NoofOpens, Min(opens.openDate) as Min_OpenDate, MAX(opens.openDate) as Max_OpenDate,
       COUNT(DISTINCT clicks.trackingId) as NoofClicks,Min(clicks.clickDate) as Min_ClickDate, MAX(clicks.clickDate) as Max_ClickDate, 
       'marketo' as data_source
FROM dev_sources_marketo.raw_campaignssends sends
LEFT JOIN dev_sources_marketo.raw_campaignopens opens ON opens.deliveryLogId = sends.deliveryLogId
LEFT JOIN dev_sources_marketo.raw_campaignsclicks clicks ON clicks.deliveryLogId = sends.deliveryLogId
GROUP BY sends.countryCode,sends.deliveryLogId,sends.deliveryID,sends.contactpersonOhubId,sends.operatorOhubId,sends.campaignLabel,sends.deliveryLabel,
         sends.sentDate
""")
marketo_campaignresponsedata = marketo_campaignresponsedata.withColumn("isOpenorClick", when( (col("NoofOpens") > 0) | (col("NoofClicks") > 0),1).otherwise(0))
marketo_campaignresponsedata = marketo_campaignresponsedata.withColumn("isNewsletter", lit(1))

In [0]:
marketo_campaignresponsedata        = marketo_campaignresponsedata.select("countryCode", "broadLogId", "contactPersonOhubId", "waveName", "sendDate", "NoofOpens", "NoofClicks", "Min_OpenDate", "Max_OpenDate", "Min_ClickDate", "Max_ClickDate","data_source","isOpenorClick","isNewsletter")

In [0]:
marketo_campaignresponsedata.write.mode("overwrite").saveAsTable(destination_table)

In [0]:
%sql
SELECT COUNT(DISTINCT broadLogId) as NoofDeliveries, 
       MIN(sendDate) as mindate
FROM data_acm.marketo_campaign_response_data_processed
WHERE contactPersonOhubId = '0a327b82-5822-4c8a-b7c7-0ce0a76945a6'

NoofDeliveries,mindate
1,2020-06-18T04:31:05.000+0000
